In [1]:
import nfl_data_py as nfl
import pandas as pd
import os
import urllib.request
import matplotlib.pyplot as plt
from matplotlib.offsetbox import AnnotationBbox
from matplotlib.offsetbox import OffsetImage
from PIL import Image
import numpy as np
from io import BytesIO
import requests
# ML Libraries
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
#import hvplot.pandas
import plotly.express as px

# Graphing 
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.pyplot import figure

In [2]:
#Import Data
nfl_data = nfl.import_schedules([2023])
original_df = nfl_data.groupby('week')


In [3]:
# Select relevant columns for team vs. team DataFrame
team_vs_team_df = nfl_data[['week', 'away_team', 'away_score', 'home_team', 'home_score']]


In [4]:
# Get data from games
weekly_nfl = nfl.import_weekly_data([2023])
weekly_nfl.columns

Downcasting floats.


Index(['player_id', 'player_name', 'player_display_name', 'position',
       'position_group', 'headshot_url', 'recent_team', 'season', 'week',
       'season_type', 'opponent_team', 'completions', 'attempts',
       'passing_yards', 'passing_tds', 'interceptions', 'sacks', 'sack_yards',
       'sack_fumbles', 'sack_fumbles_lost', 'passing_air_yards',
       'passing_yards_after_catch', 'passing_first_downs', 'passing_epa',
       'passing_2pt_conversions', 'pacr', 'dakota', 'carries', 'rushing_yards',
       'rushing_tds', 'rushing_fumbles', 'rushing_fumbles_lost',
       'rushing_first_downs', 'rushing_epa', 'rushing_2pt_conversions',
       'receptions', 'targets', 'receiving_yards', 'receiving_tds',
       'receiving_fumbles', 'receiving_fumbles_lost', 'receiving_air_yards',
       'receiving_yards_after_catch', 'receiving_first_downs', 'receiving_epa',
       'receiving_2pt_conversions', 'racr', 'target_share', 'air_yards_share',
       'wopr', 'special_teams_tds', 'fantasy_points

In [5]:
#clean up data to show data columns for each game
columns = [ 'week','recent_team','opponent_team','passing_epa',
           'passing_yards', 'rushing_epa','rushing_yards','receiving_epa'
           ]


In [6]:
#create Df
games_df = weekly_nfl[columns]
games_df.head()

,week,recent_team,opponent_team,passing_epa,passing_yards,rushing_epa,rushing_yards,receiving_epa
0,1,NYJ,BUF,-2.03196,0.0,NaN,0.0,NaN
1,4,CHI,DEN,NaN,0.0,NaN,0.0,0.483465
2,7,CHI,LV,NaN,0.0,NaN,0.0,1.437224
3,11,CHI,DET,NaN,0.0,NaN,0.0,-0.547367
4,14,CHI,DET,NaN,0.0,NaN,0.0,-0.190257


In [7]:
# Groupby team and week offense stats
team_by_week = games_df.groupby(['week', 'recent_team'])
team_by_week.head()
team_stats_per_game = team_by_week.sum()
team_stats_per_game.index.set_names(['week', 'team_offense'], inplace=True)
team_stats_per_game = team_stats_per_game.reset_index()
#team_stats_per_game_off = team_stats_per_game.drop(columns='index')

team_stats_per_game_off = team_stats_per_game.rename(columns={'team_defense': 'team_offense'})
# List of columns to add '_def' to
columns_to_suffix = ['passing_epa',
           'passing_yards', 'rushing_epa','rushing_yards','receiving_epa']

team_stats_per_game_off = team_stats_per_game_off.rename(columns={col: col + '_offense' for col in columns_to_suffix})

team_stats_per_game_off.tail()


,week,team_offense,passing_epa_offense,passing_yards_offense,rushing_epa_offense,rushing_yards_offense,receiving_epa_offense
565,21,DET,6.998413,273.0,2.268643,182.0,10.135550
566,21,KC,7.783350,241.0,-8.881944,89.0,12.629683
567,21,SF,6.624600,267.0,7.201090,155.0,9.344954
568,22,KC,6.061147,333.0,-7.526628,130.0,13.647686
569,22,SF,9.144471,276.0,-9.394331,110.0,12.717166


In [8]:
# Groupby team and week defense stats
team_by_week_def = games_df.groupby(['week', 'opponent_team'])
team_by_week_def.head()
team_stats_per_game_def = team_by_week_def.sum()

# List of columns to add '_def' to
columns_to_suffix_def = ['passing_epa',
           'passing_yards', 'rushing_epa','rushing_yards','receiving_epa']

# Add '_def' suffix to the specified columns
team_stats_per_game_def = team_stats_per_game_def.rename(columns={col: col + '_def' for col in columns_to_suffix_def})
team_stats_per_game_def.index.set_names(['week', 'team_defense'], inplace=True)
team_stats_per_game_def = team_stats_per_game_def.reset_index()


In [9]:
team_stats_per_game_def.tail()

,week,team_defense,passing_epa_def,passing_yards_def,rushing_epa_def,rushing_yards_def,receiving_epa_def
565,21,DET,6.624600,267.0,7.201090,155.0,9.344954
566,21,KC,-5.030445,272.0,2.078561,81.0,0.962597
567,21,SF,6.998413,273.0,2.268643,182.0,10.135550
568,22,KC,9.144471,276.0,-9.394331,110.0,12.717166
569,22,SF,6.061147,333.0,-7.526628,130.0,13.647686


In [10]:
# merge both dfs so it shows offense and defense stats per game
team_total_df = team_stats_per_game.merge(team_stats_per_game_def, left_on='team_offense', right_on='team_defense')
# Filter rows where 'team_offense' is equal to 'team_defense'
team_total_df = team_total_df[team_total_df['team_offense'] == team_total_df['team_defense']]
#remove duplicates
team_total_df = team_total_df[(team_total_df['team_offense'] == team_total_df['team_defense']) & (team_total_df['week_x'] == team_total_df['week_y'])]
team_total_df.head(1)

,week_x,team_offense,passing_epa,passing_yards,rushing_epa,rushing_yards,receiving_epa,week_y,team_defense,passing_epa_def,passing_yards_def,rushing_epa_def,rushing_yards_def,receiving_epa_def
0,1,ARI,-9.239668,132.0,-11.179296,96.0,-2.79934,1,ARI,-13.413063,202.0,-5.708867,92.0,1.409773


In [11]:
# create a copy of team_total_df for home team and away team

# Make the first copy of the original DataFrame
home_team_df = team_total_df.copy()

# Make a second copy from the first copy
away_team_df = home_team_df.copy()

In [12]:
#make a home_team df
# Add '_home' to each column name
for column in home_team_df.columns:
    new_column_name_home = column + '_home'
    home_team_df.rename(columns={column: new_column_name_home}, inplace=True)
    
home_team_df.head()

,week_x_home,team_offense_home,passing_epa_home,passing_yards_home,rushing_epa_home,rushing_yards_home,receiving_epa_home,week_y_home,team_defense_home,passing_epa_def_home,passing_yards_def_home,rushing_epa_def_home,rushing_yards_def_home,receiving_epa_def_home
0,1,ARI,-9.239668,132.0,-11.179296,96.0,-2.799340,1,ARI,-13.413063,202.0,-5.708867,92.0,1.409773
18,2,ARI,7.430885,228.0,2.366217,151.0,7.653436,2,ARI,8.180348,321.0,6.719742,127.0,11.507128
36,3,ARI,4.560631,189.0,7.001422,222.0,8.408302,3,ARI,-3.659615,249.0,8.314601,185.0,0.522374
54,4,ARI,3.898311,265.0,5.478536,105.0,6.421503,4,ARI,22.844091,283.0,2.675849,124.0,24.983665
72,5,ARI,-15.651534,166.0,-1.152939,142.0,-5.404606,5,ARI,9.498447,317.0,-8.893337,93.0,16.416182


In [13]:
#make a away_team df
# Add '_away' to each column name
for column in away_team_df.columns:
    new_column_name_away = column + '_away'
    away_team_df.rename(columns={column: new_column_name_away}, inplace=True)
away_team_df.head()

,week_x_away,team_offense_away,passing_epa_away,passing_yards_away,rushing_epa_away,rushing_yards_away,receiving_epa_away,week_y_away,team_defense_away,passing_epa_def_away,passing_yards_def_away,rushing_epa_def_away,rushing_yards_def_away,receiving_epa_def_away
0,1,ARI,-9.239668,132.0,-11.179296,96.0,-2.799340,1,ARI,-13.413063,202.0,-5.708867,92.0,1.409773
18,2,ARI,7.430885,228.0,2.366217,151.0,7.653436,2,ARI,8.180348,321.0,6.719742,127.0,11.507128
36,3,ARI,4.560631,189.0,7.001422,222.0,8.408302,3,ARI,-3.659615,249.0,8.314601,185.0,0.522374
54,4,ARI,3.898311,265.0,5.478536,105.0,6.421503,4,ARI,22.844091,283.0,2.675849,124.0,24.983665
72,5,ARI,-15.651534,166.0,-1.152939,142.0,-5.404606,5,ARI,9.498447,317.0,-8.893337,93.0,16.416182


In [14]:
# Make df with points and stats

# merge home team with home stats
home_team = team_vs_team_df.merge(home_team_df, left_on='home_team', right_on='team_offense_home')
#remove duplicates
home_team = home_team[(home_team['home_team'] ==  home_team['team_offense_home']) & (home_team['week_x_home'] == home_team['week'])]
home_team.tail(10)

,week,away_team,away_score,home_team,home_score,week_x_home,team_offense_home,passing_epa_home,passing_yards_home,rushing_epa_home,rushing_yards_home,receiving_epa_home,week_y_home,team_defense_home,passing_epa_def_home,passing_yards_def_home,rushing_epa_def_home,rushing_yards_def_home,receiving_epa_def_home
4944,18,BUF,21.0,MIA,14.0,18,MIA,-0.179888,173.0,2.018006,108.0,1.253841,18,MIA,3.935818,359.0,1.620589,128.0,11.626987
4948,3,PIT,23.0,LV,18.0,3,LV,-8.352474,324.0,-2.516379,69.0,-2.013242,3,LV,5.921596,235.0,-6.737266,105.0,9.555780
4967,5,GB,13.0,LV,17.0,5,LV,-1.803250,208.0,-3.381476,96.0,6.604667,5,LV,-14.767113,182.0,4.087527,110.0,-11.481108
4985,6,NE,17.0,LV,21.0,6,LV,9.882565,264.0,-10.972410,84.0,11.559125,6,LV,-4.471819,200.0,4.164759,83.0,3.130720
5005,9,NYG,6.0,LV,30.0,9,LV,7.603049,209.0,-7.482466,125.0,9.152975,9,LV,-19.027122,200.0,-4.605570,123.0,-7.132212
5023,10,NYJ,12.0,LV,16.0,10,LV,-9.807395,153.0,-1.637336,148.0,-3.269739,10,LV,-4.556659,263.0,-3.730515,108.0,0.062029
5042,12,KC,31.0,LV,17.0,12,LV,3.499284,248.0,-5.031726,123.0,6.922385,12,LV,12.587396,298.0,-0.940888,69.0,15.450614
5060,14,MIN,3.0,LV,0.0,14,LV,-19.203550,171.0,-0.846225,56.0,-15.269022,14,LV,-12.121390,146.0,-2.298960,132.0,0.565894
5078,15,LAC,21.0,LV,63.0,15,LV,12.323879,260.0,3.581412,124.0,16.015894,15,LV,-10.320502,257.0,-13.295756,92.0,7.879056
5098,18,DEN,14.0,LV,27.0,18,LV,9.306783,244.0,-1.255764,129.0,17.544756,18,LV,0.353189,272.0,-2.951732,48.0,9.411932


In [15]:
# merge away team with home stats
away_team = home_team.merge(away_team_df, left_on='away_team', right_on='team_offense_away')
#remove duplicates
away_team = away_team[(away_team['away_team'] ==  away_team['team_offense_away']) & (away_team['week_x_away'] == away_team['week'])]
away_team.head(10)

,week,away_team,away_score,home_team,home_score,week_x_home,team_offense_home,passing_epa_home,passing_yards_home,rushing_epa_home,...,rushing_epa_away,rushing_yards_away,receiving_epa_away,week_y_away,team_defense_away,passing_epa_def_away,passing_yards_def_away,rushing_epa_def_away,rushing_yards_def_away,receiving_epa_def_away
0,1,DET,21.0,KC,20.0,1,KC,-6.453613,226.0,-2.581925,...,-3.636585,118.0,6.054341,1,DET,-6.453613,226.0,-2.581925,90.0,-5.052831
26,7,DET,6.0,BAL,38.0,7,BAL,24.334782,357.0,1.219639,...,1.834399,84.0,0.542692,7,DET,24.334782,357.0,1.219639,146.0,24.334782
54,16,DET,30.0,MIN,24.0,16,MIN,5.691321,411.0,-4.351473,...,-0.709332,143.0,16.405670,16,DET,5.691321,411.0,-4.351473,17.0,13.313128
71,13,DET,33.0,NO,28.0,13,NO,6.018938,267.0,-8.409609,...,-1.068709,142.0,10.316429,13,DET,6.018938,267.0,-8.409609,113.0,10.585057
92,14,DET,13.0,CHI,28.0,14,CHI,2.945851,223.0,-0.283723,...,-2.966777,140.0,0.374628,14,DET,2.945851,223.0,-0.283723,142.0,10.205959
108,10,DET,41.0,LAC,38.0,10,LAC,14.587980,323.0,-0.869600,...,11.664154,200.0,19.774326,10,DET,14.587980,323.0,-0.869600,98.0,14.587980
125,6,DET,20.0,TB,6.0,6,TB,-8.300112,206.0,-4.832714,...,-7.840065,40.0,22.679295,6,DET,-8.300112,206.0,-4.832714,46.0,-7.366295
155,17,DET,19.0,DAL,20.0,17,DAL,11.299153,345.0,-7.875702,...,-5.493361,125.0,5.870264,17,DET,11.299153,345.0,-7.875702,61.0,7.770870
179,21,DET,31.0,SF,34.0,21,SF,6.624600,267.0,7.201090,...,2.268643,182.0,10.135550,21,DET,6.624600,267.0,7.201090,155.0,9.344954
183,4,DET,34.0,GB,20.0,4,GB,-12.693180,246.0,0.423209,...,4.333835,211.0,6.932714,4,DET,-12.693180,246.0,0.423209,27.0,-5.993364


In [16]:
# rename df
full_df = away_team.copy()

In [17]:
#full_df.to_excel('training_nfl.xlsx', index=False)


In [18]:
# Create Columns for training
column_train_x = [
   'passing_epa_home','passing_yards_home','rushing_epa_home','rushing_yards_home','receiving_epa_home',
    'passing_epa_def_home','passing_yards_def_home','rushing_epa_def_home','rushing_yards_def_home','receiving_epa_def_home',
       'passing_epa_away','passing_yards_away','rushing_epa_away','rushing_yards_away','receiving_epa_away',
           'passing_epa_def_away','passing_yards_def_away','rushing_epa_def_away','rushing_yards_def_away','receiving_epa_def_away',


]




column_train_y = ['home_score','away_score']

In [19]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
# model
# Split data into features (X) and target variable (y)
X = full_df[column_train_x]
y = full_df[column_train_y]

# Encode categorical variables using one-hot encoding if needed

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

# Create a Linear Regression model
model = LinearRegression()

# Train the model
model.fit(X_train, y_train)

# Make predictions
predictions = model.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
rmse = mean_squared_error(y_test, predictions, squared=False)
r2 = r2_score(y_test, predictions)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R-squared (R2): {r2}")


Mean Absolute Error (MAE): 4.25987992788616
Mean Squared Error (MSE): 27.654206553979552
Root Mean Squared Error (RMSE): 5.248728600356156
R-squared (R2): 0.7536782661853927


In [20]:
#get 2024 averages
#create Df
weekly_nfl_two = nfl.import_weekly_data([2024])

games_df_two = weekly_nfl_two[columns]
games_df_two.head()

# Groupby team and week offense stats
team_by_week_two = games_df_two.groupby(['week', 'recent_team'])
team_by_week_two.head()
team_stats_per_game_two = team_by_week_two.sum()
team_stats_per_game_two.index.set_names(['week','team_offense'], inplace=True)
team_stats_per_game_two = team_stats_per_game_two.reset_index()
#team_stats_per_game_off = team_stats_per_game.drop(columns='index')

team_stats_per_game_off_two = team_stats_per_game_two.rename(columns={'team_defense': 'team_offense'})
# List of columns to add '_def' to
columns_to_suffix = ['passing_epa',
           'passing_yards', 'rushing_epa','rushing_yards','receiving_epa']

team_stats_per_game_off_two = team_stats_per_game_off_two.rename(columns={col: col + '_offense' for col in columns_to_suffix})

team_stats_per_game_off_two.head()



Downcasting floats.


,week,team_offense,passing_epa_offense,passing_yards_offense,rushing_epa_offense,rushing_yards_offense,receiving_epa_offense
0,1,ARI,-1.184850,162.0,3.931950,124.0,8.129718
1,1,ATL,-11.084852,155.0,-6.224773,89.0,-7.576805
2,1,BAL,-2.491160,273.0,8.459208,185.0,3.723055
3,1,BUF,9.161098,232.0,3.497710,130.0,15.683219
4,1,CAR,-18.467295,161.0,-8.230616,58.0,-10.917299


In [21]:
# add defense stats for 2024

# Groupby team and week defense stats
team_by_week_def_two = games_df_two.groupby(['week', 'opponent_team'])
team_by_week_def_two.head()
team_stats_per_game_def_two = team_by_week_def_two.sum()

# List of columns to add '_def' to
columns_to_suffix_def = ['passing_epa',
           'passing_yards', 'rushing_epa','rushing_yards','receiving_epa']

# Add '_def' suffix to the specified columns
team_stats_per_game_def_two = team_stats_per_game_def_two.rename(columns={col: col + '_def' for col in columns_to_suffix_def})
team_stats_per_game_def_two.index.set_names(['week', 'team_defense'], inplace=True)
team_stats_per_game_def_two = team_stats_per_game_def_two.reset_index()


In [23]:
### Create mean of these stats to create test data
#home team
mean_offense_two = team_stats_per_game_off_two.groupby('team_offense').mean()
mean_offense_two = mean_offense_two.drop('week', axis=1)
mean_offense_two = mean_offense_two.reset_index()

mean_offense_two.head()

,team_offense,passing_epa_offense,passing_yards_offense,rushing_epa_offense,rushing_yards_offense,receiving_epa_offense
0,ARI,3.915058,216.916672,0.601972,141.416672,5.574213
1,ATL,2.636270,257.500000,-1.914354,117.750000,6.993577
2,BAL,9.803666,253.153839,2.494580,179.076920,12.680863
3,BUF,6.594079,225.000000,3.176729,127.583336,9.452875
4,CAR,-4.987714,197.500000,-0.596101,105.250000,-1.800003


In [24]:
#defense team
# Split the DataFrame into non-numerical and numerical columns
mean_defense_two = team_stats_per_game_def_two.groupby('team_defense').mean()
# Remove the week column
mean_defense_two = mean_defense_two.drop('week', axis=1)
mean_defense_two = mean_defense_two.reset_index()

mean_defense_two.head()


,team_defense,passing_epa_def,passing_yards_def,rushing_epa_def,rushing_yards_def,receiving_epa_def
0,ARI,2.860660,235.166672,0.241055,117.333336,8.520907
1,ATL,3.252783,228.500000,-0.430651,121.250000,6.171782
2,BAL,4.396358,283.615387,-2.587396,82.692307,10.561145
3,BUF,-1.420078,215.750000,-1.740077,121.916664,5.485683
4,CAR,5.494976,237.333328,2.914903,166.833328,7.860812


In [25]:
# Merge on defense vs offense per team
test_data_df = pd.merge(mean_defense_two, mean_offense_two, left_on='team_defense',right_on='team_offense', how='left')
test_data_df.head()

,team_defense,passing_epa_def,passing_yards_def,rushing_epa_def,rushing_yards_def,receiving_epa_def,team_offense,passing_epa_offense,passing_yards_offense,rushing_epa_offense,rushing_yards_offense,receiving_epa_offense
0,ARI,2.860660,235.166672,0.241055,117.333336,8.520907,ARI,3.915058,216.916672,0.601972,141.416672,5.574213
1,ATL,3.252783,228.500000,-0.430651,121.250000,6.171782,ATL,2.636270,257.500000,-1.914354,117.750000,6.993577
2,BAL,4.396358,283.615387,-2.587396,82.692307,10.561145,BAL,9.803666,253.153839,2.494580,179.076920,12.680863
3,BUF,-1.420078,215.750000,-1.740077,121.916664,5.485683,BUF,6.594079,225.000000,3.176729,127.583336,9.452875
4,CAR,5.494976,237.333328,2.914903,166.833328,7.860812,CAR,-4.987714,197.500000,-0.596101,105.250000,-1.800003


In [26]:
# create DF for upcoming week
# create stats to show upcpoming games

#2024 update to get 2024 schedule
nfl_data_two = nfl.import_schedules([2024])

grouped = nfl_data_two.groupby('week')
# Select the group corresponding to week 6
week_data = grouped.get_group(14)
prior_week = grouped.get_group(1)
get_teams =['home_team','away_team','home_score','away_score']
schedule_week_df = week_data[get_teams]
schedule_week_df.reset_index(drop=True, inplace=True)
schedule_week_df.head()

,home_team,away_team,home_score,away_score
0,DET,GB,NaN,NaN
1,MIA,NYJ,NaN,NaN
2,MIN,ATL,NaN,NaN
3,NYG,NO,NaN,NaN
4,PHI,CAR,NaN,NaN


In [27]:
# create a copy of team_total_df for home team and away team for testing

# Make the first copy of the original DataFrame
home_team_test = test_data_df.copy()

# Make a second copy from the first copy
away_team_test = home_team_test.copy()

In [28]:
# Add '_home' to each column name
for column in home_team_test.columns:
    new_column_name_test = column + '_home'
    home_team_test.rename(columns={column: new_column_name_test}, inplace=True)
    
home_team_test.head()

,team_defense_home,passing_epa_def_home,passing_yards_def_home,rushing_epa_def_home,rushing_yards_def_home,receiving_epa_def_home,team_offense_home,passing_epa_offense_home,passing_yards_offense_home,rushing_epa_offense_home,rushing_yards_offense_home,receiving_epa_offense_home
0,ARI,2.860660,235.166672,0.241055,117.333336,8.520907,ARI,3.915058,216.916672,0.601972,141.416672,5.574213
1,ATL,3.252783,228.500000,-0.430651,121.250000,6.171782,ATL,2.636270,257.500000,-1.914354,117.750000,6.993577
2,BAL,4.396358,283.615387,-2.587396,82.692307,10.561145,BAL,9.803666,253.153839,2.494580,179.076920,12.680863
3,BUF,-1.420078,215.750000,-1.740077,121.916664,5.485683,BUF,6.594079,225.000000,3.176729,127.583336,9.452875
4,CAR,5.494976,237.333328,2.914903,166.833328,7.860812,CAR,-4.987714,197.500000,-0.596101,105.250000,-1.800003


In [29]:
# Add '_away' to each column name
for column in away_team_test.columns:
    new_column_name_test_away = column + '_away'
    away_team_test.rename(columns={column: new_column_name_test_away}, inplace=True)
    
away_team_test.head()

,team_defense_away,passing_epa_def_away,passing_yards_def_away,rushing_epa_def_away,rushing_yards_def_away,receiving_epa_def_away,team_offense_away,passing_epa_offense_away,passing_yards_offense_away,rushing_epa_offense_away,rushing_yards_offense_away,receiving_epa_offense_away
0,ARI,2.860660,235.166672,0.241055,117.333336,8.520907,ARI,3.915058,216.916672,0.601972,141.416672,5.574213
1,ATL,3.252783,228.500000,-0.430651,121.250000,6.171782,ATL,2.636270,257.500000,-1.914354,117.750000,6.993577
2,BAL,4.396358,283.615387,-2.587396,82.692307,10.561145,BAL,9.803666,253.153839,2.494580,179.076920,12.680863
3,BUF,-1.420078,215.750000,-1.740077,121.916664,5.485683,BUF,6.594079,225.000000,3.176729,127.583336,9.452875
4,CAR,5.494976,237.333328,2.914903,166.833328,7.860812,CAR,-4.987714,197.500000,-0.596101,105.250000,-1.800003


In [30]:
# Make df with points and stats for testing upcoming week

# merge home team with home stats
testing_home = schedule_week_df.merge(home_team_test, left_on='home_team', right_on='team_offense_home')
#remove duplicates
#testing_home = home_team[(home_team['home_team'] ==  home_team['team_offense_home']) & (home_team['week_x_home'] == home_team['week'])]
testing_home.head(1)

,home_team,away_team,home_score,away_score,team_defense_home,passing_epa_def_home,passing_yards_def_home,rushing_epa_def_home,rushing_yards_def_home,receiving_epa_def_home,team_offense_home,passing_epa_offense_home,passing_yards_offense_home,rushing_epa_offense_home,rushing_yards_offense_home,receiving_epa_offense_home
0,DET,GB,NaN,NaN,DET,-4.571522,243.666672,-1.763825,93.5,-1.732599,DET,7.79464,255.916672,1.092132,154.416672,11.954906


In [31]:
# merge away team with home stats
testing_away = testing_home.merge(away_team_test, left_on='away_team', right_on='team_offense_away')
#remove duplicates
#away_team = away_team[(away_team['away_team'] ==  away_team['team_offense_away']) & (away_team['week_x_away'] == away_team['week'])]
testing_away.head(10)

,home_team,away_team,home_score,away_score,team_defense_home,passing_epa_def_home,passing_yards_def_home,rushing_epa_def_home,rushing_yards_def_home,receiving_epa_def_home,...,passing_yards_def_away,rushing_epa_def_away,rushing_yards_def_away,receiving_epa_def_away,team_offense_away,passing_epa_offense_away,passing_yards_offense_away,rushing_epa_offense_away,rushing_yards_offense_away,receiving_epa_offense_away
0,DET,GB,NaN,NaN,DET,-4.571522,243.666672,-1.763825,93.500000,-1.732599,...,235.416672,-1.531953,106.583336,5.289484,GB,4.713626,241.500000,0.196099,148.500000,7.103692
1,MIA,NYJ,NaN,NaN,MIA,1.167590,216.833328,-1.708784,107.500000,5.470481,...,197.916672,0.663934,126.250000,6.553015,NYJ,-0.278334,221.916672,-2.397728,87.333336,3.676316
2,MIN,ATL,NaN,NaN,MIN,-2.655221,265.750000,-4.679358,81.333336,2.451197,...,228.500000,-0.430651,121.250000,6.171782,ATL,2.636270,257.500000,-1.914354,117.750000,6.993577
3,NYG,NO,NaN,NaN,NYG,1.787248,213.250000,1.942771,145.833328,7.770610,...,263.583344,2.083805,136.000000,3.856392,NO,-0.927221,224.583328,0.914707,131.750000,2.865740
4,PHI,CAR,NaN,NaN,PHI,-1.651734,200.250000,-2.674004,104.750000,2.904828,...,237.333328,2.914903,166.833328,7.860812,CAR,-4.987714,197.500000,-0.596101,105.250000,-1.800003
5,PIT,CLE,NaN,NaN,PIT,-1.329135,237.083328,-2.074920,90.500000,4.253414,...,237.250000,-1.319246,128.583328,7.146579,CLE,-6.583254,249.416672,-1.032807,87.250000,1.109373
6,TB,LV,NaN,NaN,TB,5.008042,277.083344,-0.054781,117.166664,9.508131,...,230.916672,-2.212906,114.750000,8.283759,LV,-4.432546,248.333328,-5.782799,78.083336,4.998331
7,TEN,JAX,NaN,NaN,TEN,0.057864,186.833328,-1.338672,120.083336,5.080022,...,286.166656,0.380867,133.166672,13.819068,JAX,-1.197189,213.083328,-0.606285,101.250000,2.767776
8,ARI,SEA,NaN,NaN,ARI,2.860660,235.166672,0.241055,117.333336,8.520907,...,228.333328,-0.202858,127.000000,4.154601,SEA,1.498065,273.000000,-2.450692,88.666664,7.083121
9,LA,BUF,NaN,NaN,LA,1.836054,234.750000,-0.097042,144.166672,8.504375,...,215.750000,-1.740077,121.916664,5.485683,BUF,6.594079,225.000000,3.176729,127.583336,9.452875


In [32]:
# rename df
full_test = testing_away.copy()
#full_test = full_test.reset_index()

full_test = full_test.rename(columns={
    'passing_epa_offense_away': 'passing_epa_away',
    'passing_yards_offense_away': 'passing_yards_away',
    'receiving_epa_offense_away': 'receiving_epa_away',
    'rushing_epa_offense_away': 'rushing_epa_away',
    'rushing_yards_offense_away': 'rushing_yards_away',
     'passing_epa_offense_home': 'passing_epa_home',
    'passing_yards_offense_home': 'passing_yards_home',
    'receiving_epa_offense_home': 'receiving_epa_home',
    'rushing_epa_offense_home': 'rushing_epa_home',
    'rushing_yards_offense_home': 'rushing_yards_home',
})
full_test.dtypes

home_team                  object
away_team                  object
home_score                float64
away_score                float64
team_defense_home          object
passing_epa_def_home      float32
passing_yards_def_home    float32
rushing_epa_def_home      float32
rushing_yards_def_home    float32
receiving_epa_def_home    float32
team_offense_home          object
passing_epa_home          float32
passing_yards_home        float32
rushing_epa_home          float32
rushing_yards_home        float32
receiving_epa_home        float32
team_defense_away          object
passing_epa_def_away      float32
passing_yards_def_away    float32
rushing_epa_def_away      float32
rushing_yards_def_away    float32
receiving_epa_def_away    float32
team_offense_away          object
passing_epa_away          float32
passing_yards_away        float32
rushing_epa_away          float32
rushing_yards_away        float32
receiving_epa_away        float32
dtype: object

In [33]:
column_test_x = ['passing_epa_def_away', 'passing_yards_def_away',
       'rushing_epa_def_away', 'rushing_yards_def_away',
       'receiving_epa_def_away','passing_epa_away', 'passing_yards_away',
    'rushing_epa_away', 'rushing_yards_away','receiving_epa_away',
     'passing_epa_def_home', 'passing_yards_def_home',
       'rushing_epa_def_home', 'rushing_yards_def_home',
       'receiving_epa_def_home','passing_epa_home', 'passing_yards_home',
    'rushing_epa_home', 'rushing_yards_home',
       'receiving_epa_home',            
                ]

In [34]:
# Now, use the trained model to predict score for df_test
X_test = full_test[column_test_x]
y_pred = model.predict(X_test)

# The predictions are stored in y_pred
print("Predicted Scores:")
print(y_pred)

Predicted Scores:
[[26.65411  20.166994]
 [21.482792 18.90474 ]
 [25.565956 18.89796 ]
 [22.088934 23.466537]
 [29.219707 16.538328]
 [26.123432 17.75177 ]
 [26.574425 19.417633]
 [23.592571 19.11697 ]
 [23.87724  21.222778]
 [20.54705  24.499992]
 [24.450846 19.784584]
 [21.760849 20.27864 ]
 [21.604395 24.816807]]


/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/base.py:488: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)


In [35]:
# add predicted scores into df: 
predict_cols = ['home_team','away_team']
df_predictions = full_test[predict_cols]
scores_df = pd.DataFrame(y_pred, columns=['home_score', 'away_score'])
df_predictions = pd.concat([df_predictions, scores_df], axis=1)
df_predictions['Over/Under'] = df_predictions['home_score'] + df_predictions['away_score']
# Assuming 'home_score', 'away_score', 'home_team', and 'away_team' are columns in your DataFrame
df_predictions['Favorite'] = np.where(df_predictions['home_score'] > df_predictions['away_score'], df_predictions['home_team'], df_predictions['away_team'])
df_predictions['Spread'] = df_predictions['away_score'] - df_predictions['home_score']
df_predictions['spread'] = df_predictions['Spread'].apply(lambda x: '-' + str(x) if x >= 0 else str(x))
df_predictions = df_predictions.drop('Spread', axis=1)

df_predictions.head(17)

,home_team,away_team,home_score,away_score,Over/Under,Favorite,spread
0,DET,GB,26.654110,20.166994,46.821106,DET,-6.487115859985352
1,MIA,NYJ,21.482792,18.904739,40.387531,MIA,-2.578052520751953
2,MIN,ATL,25.565956,18.897961,44.463917,MIN,-6.667995452880859
3,NYG,NO,22.088934,23.466537,45.555473,NO,-1.377603530883789
4,PHI,CAR,29.219707,16.538328,45.758034,PHI,-12.681379318237305
5,PIT,CLE,26.123432,17.751770,43.875202,PIT,-8.371662139892578
6,TB,LV,26.574425,19.417633,45.992058,TB,-7.156791687011719
7,TEN,JAX,23.592571,19.116970,42.709541,TEN,-4.4756011962890625
8,ARI,SEA,23.877239,21.222778,45.100018,ARI,-2.654460906982422
9,LA,BUF,20.547050,24.499992,45.047043,BUF,-3.95294189453125


In [36]:
# Convert the 'spread' column to float
df_predictions['spread'] = pd.to_numeric(df_predictions['spread'], errors='coerce')
# Round the numeric columns to 1 decimal place
df_predictions['home_score'] = df_predictions['home_score'].round(2)
df_predictions['away_score'] = df_predictions['away_score'].round(2)
df_predictions['Over/Under'] = df_predictions['Over/Under'].round(2)
df_predictions['spread'] = df_predictions['spread'].round(2)
# Set display precision
pd.set_option('display.float_format', '{:.1f}'.format)



df_predictions.head(16)

,home_team,away_team,home_score,away_score,Over/Under,Favorite,spread
0,DET,GB,26.6,20.2,46.8,DET,-6.5
1,MIA,NYJ,21.5,18.9,40.4,MIA,-2.6
2,MIN,ATL,25.6,18.9,44.5,MIN,-6.7
3,NYG,NO,22.1,23.5,45.6,NO,-1.4
4,PHI,CAR,29.2,16.5,45.8,PHI,-12.7
5,PIT,CLE,26.1,17.8,43.9,PIT,-8.4
6,TB,LV,26.6,19.4,46.0,TB,-7.2
7,TEN,JAX,23.6,19.1,42.7,TEN,-4.5
8,ARI,SEA,23.9,21.2,45.1,ARI,-2.6
9,LA,BUF,20.5,24.5,45.0,BUF,-4.0


In [37]:
#df_predictions.to_excel('week_1_pred.xlsx', index=False)


In [38]:
# Add in actual odds from these games
vegas_df = pd.read_excel('vegas_odds/week_one_vegas_odds.xlsx')
vegas_df.head(16)


,home_team,away_team,Over/Under_Vegas,Favorite_Vegas,Spread_Vegas
0,KC,BAL,46.0,KC,-3.0
1,PHI,GB,48.5,PHI,-2.5
2,CIN,NE,40.5,CIN,-8.5
3,CHI,TEN,45.0,CHI,-3.5
4,MIA,JAX,49.5,MIA,-3.0
5,IND,HOU,49.0,HOU,-2.5
6,BUF,ARI,48.0,BUF,-6.0
7,ATL,PIT,42.0,ATL,-3.0
8,NYG,MIN,41.5,MIN,-1.5
9,NO,CAR,41.5,NO,-4.0


In [39]:
#merge so model and vegas odds are on each game
# Merge DataFrames
model_vegas_df = pd.merge(df_predictions, vegas_df, on=['home_team', 'away_team'])
model_vegas_df['Over/Under_Difference'] = model_vegas_df['Over/Under'] - model_vegas_df['Over/Under_Vegas']


model_vegas_df.head(16)


,home_team,away_team,home_score,away_score,Over/Under,Favorite,spread,Over/Under_Vegas,Favorite_Vegas,Spread_Vegas,Over/Under_Difference


In [147]:
from flask import Flask, render_template
app = Flask(__name__)

@app.route('/')
def display_dataframe():
    return df_predictions.to_html()

if __name__ == '__main__':
    app.run()










 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [06/Sep/2024 15:44:53] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Sep/2024 15:44:54] "GET /favicon.ico HTTP/1.1" 404 -
